In [10]:
# Downloading necessary files
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import mplfinance as mpf 
import matplotlib.pyplot as plt
import datetime as dt
import datetime
from datetime import datetime
import plotly.graph_objects as go
import re
import json
import requests 
import codecs
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.offline as pyo


# display plotly in browser when run in Spyder
# pio.renderers.default = 'browser'

In [11]:
# stock =  input('Type in the stock symbol: ').upper()
stock = 'LXRX'

In [12]:
def create_df():

    # =============================================================================
    # Creating a new dataset
    # =============================================================================
    
    global df
    
    start = "2009-01-01"
    end = dt.datetime.now()
    short_sma = 50
    long_sma = 200
    
    # creating a dataset for selected stock 
    df = yf.download(stock, start, end,interval = '1d')
    df = pd.DataFrame(df.dropna(), columns = ['Open', 'High','Low','Close', 'Adj Close', 'Volume'])


    # preparing data from time series analysis
    # eliminating any NAs 
    df.index = pd.to_datetime(df.index)
    df = df.asfreq('B')

    print('============================================================')
    print('Nan in each columns before Backfill for' , stock)
    print('------------------------------------------------------------')  
    print(df.isna().sum())
    print('The df has {} rows and {} columns.'.format(*df.shape))
    
    df = df.bfill()
    print('============================================================')
    print('Nan in each columns after Backfill for' , stock)
    print('------------------------------------------------------------')  
    print(df.isna().sum())
    print('The df has {} rows and {} columns.'.format(*df.shape))
#    df = df.dropna()
    
    # Create short SMA
    df['short_SMA'] = df.iloc[:,1].rolling(window = short_sma).mean()
    
    # Create Long SMA
    df['long_SMA'] = df.iloc[:,1].rolling(window = long_sma).mean()
    
    # Create daily_return
    df['daily_return'] = df['Close'].pct_change(periods=1).mul(100)
    
    # Create monthly_return
    df['monthly_return'] = df['Close'].pct_change(periods=30).mul(100)
    
    # Create annual_return
    df['annual_return'] = df['Close'].pct_change(periods=365).mul(100)
    df['Name'] = stock

    print('============================================================')
    print(stock, '- Full Dataset')
    print('------------------------------------------------------------')
    print(df.head())
    print('------------------------------------------------------------')
    print(stock, 'Full Dataset - Column Names')
    print(df.columns)

    print('============================================================')
    print('Nan in each columns after Backfill for' , stock)
    print('------------------------------------------------------------')  
    print(df.isna().sum())
    print('============================================================')
    
    # write to csv
    df.to_csv(r"df.csv", index =  True)
    

In [13]:
create_df()

[*********************100%***********************]  1 of 1 completed
Nan in each columns before Backfill for LXRX
------------------------------------------------------------
Open         116
High         116
Low          116
Close        116
Adj Close    116
Volume       116
dtype: int64
The df has 3364 rows and 6 columns.
Nan in each columns after Backfill for LXRX
------------------------------------------------------------
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
The df has 3364 rows and 6 columns.
LXRX - Full Dataset
------------------------------------------------------------
             Open   High    Low  Close  Adj Close   Volume  short_SMA  \
Date                                                                    
2009-01-02   9.80  10.64   9.03  10.08      10.08  26943.0        NaN   
2009-01-05  10.15  10.43   8.82  10.29      10.29  16286.0        NaN   
2009-01-06  10.36  11.13  10.29  11.06      11.06  11800.

In [14]:
def create_y():
    
    global y
    
    y = pd.DataFrame(df['Close'], columns = ['Close'])
    y.sort_index(inplace = True)
    # dropping the first na (because there is no difference)
    y = y.dropna()
    y['Name'] = stock

    # Create SMA 16-Period for MACD
    y['SMA_12'] = df.iloc[:,1].rolling(window = 12).mean()
    
    # Create SMA 16-Period for MACD
    y['SMA_26'] = df.iloc[:,1].rolling(window = 26).mean()
    
    y['MACD'] = y['SMA_12'] -  y['SMA_26']
    
    # examining the pct_change
    y['Close Percentage Change'] = y['Close'].pct_change(1)
    
    # Creating a new variable, examining the difference for each observation
    y['diff'] = y['Close'].diff()

    # logging the target varialbe due to great variance
    y['log_Close'] = np.log(y['Close'])
    
    # Creating a new variable, examining the difference for each observation
    y['log_Close_diff'] = y['log_Close'].diff()
    
    y['Logged Close Percentage Change'] = y['log_Close'].pct_change(1)
    
    # dropping the first na (because there is no difference)
    y = y.dropna()
    

#    # write to csv
#    y.to_csv(r"y.csv", index =  True)

    print('============================================================')
    print(stock, '- Target Variable')
    print('------------------------------------------------------------')
    print(y.head())
    print('------------------------------------------------------------')
    print('Column Names')
    print(y.columns)
    print('============================================================')
    


In [15]:
create_y()

LXRX - Target Variable
------------------------------------------------------------
            Close  Name     SMA_12     SMA_26      MACD  \
Date                                                      
2009-02-06  11.55  LXRX   9.823334  10.098846 -0.275513   
2009-02-09  11.90  LXRX  10.085833  10.147308 -0.061474   
2009-02-10  10.78  LXRX  10.377500  10.217308  0.160192   
2009-02-11  10.15  LXRX  10.535000  10.211923  0.323077   
2009-02-12  10.43  LXRX  10.663333  10.201154  0.462179   

            Close Percentage Change      diff  log_Close  log_Close_diff  \
Date                                                                       
2009-02-06                 0.078431  0.840000   2.446685        0.075508   
2009-02-09                 0.030303  0.349999   2.476538        0.029853   
2009-02-10                -0.094118 -1.120000   2.377693       -0.098846   
2009-02-11                -0.058442 -0.630000   2.317474       -0.060219   
2009-02-12                 0.027586  0.280001 

In [ ]:
def price_sma_volume_chart():
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=[
            'Price and Death Cross of {}'.format(str(stock)),
            'Volume of {}'.format(str(stock))])

    # Lineplots of price and moving averages
    fig.add_trace(go.Scatter(
                            x = df.index,
                            y = df['Close'],
                            name = stock,
                            mode='lines',
                            customdata = df['Name'],
                            hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Closing Price: %{y:$,.2f}<br>" ,
                            line = dict(color="black")), row = 1, col = 1)

    fig.add_trace(go.Scatter(x = df.index,
                             y = df['short_SMA'],
                             name = 'Short SMA 50-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Short (50-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="red")), row = 1, col = 1)

    fig.add_trace(go.Scatter(x = df.index,
                             y = df['long_SMA'],
                             name = 'Long SMA 200-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Long (200-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="green")), row = 1, col = 1)
    # Barplot of volume
    fig.add_trace(go.Bar(x = df.index,
                    y = df['Volume'],
                    name = 'Volume',
                    customdata = df['Name'],
                    hovertemplate="<b>%{customdata}</b><br><br>" +
                                    "Date: %{x|%d %b %Y} <br>" +
                                    "Volume: %{y:,.}<br>" +
                                    "<extra></extra>",
                    marker = dict(color="black", opacity = True)), row = 2, col = 1)
    # Add title
    fig.update_layout(
            title = 'Summary of {}'.format(str(stock)),
            title_font_size = 16)
    
    
    fig['layout']['yaxis1']['title']='US Dollars'
    fig['layout']['yaxis2']['title']='Volume'
    # X-Axes
    fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD")])))
    fig.update_layout(xaxis_rangeslider_visible = False)
    fig.update_yaxes(tickprefix = '$', tickformat = ',.', row = 1, col = 1)
    #time buttons
    fig.update_xaxes(rangeselector = {'visible' :False}, row = 2, col = 1)
#     #Show
#     fig.show()
    pyo.iplot(fig)


In [ ]:
def candlestick_moving_average():

    fig  = go.Figure()

    trace1 = go.Candlestick(
        x = df.index,
        open = df["Open"],
        high = df["High"],
        low = df["Low"],
        close = df["Close"],
        name = stock)

    data = [trace1]

    for i in range(5, 201, 5):

        sma = go.Scatter(
            x = df.index,
            y = df["Close"].rolling(i).mean(), # Pandas SMA
            name = "SMA" + str(i),
            line = dict(color = "#3E86AB",width=3),
            customdata = df['Name'],
            hovertemplate="<b>%{customdata}</b><br><br>" +
                        "Date: %{x|%d %b %Y} <br>" +
                        "Simple Moving Average Price: %{y:$,.2f}<br>",
            opacity = 0.7,
            visible = False,
        )

        data.append(sma)

    sliders = dict(

        # GENERAL
        steps = [],
        currentvalue = dict(
            font = dict(size = 16),
            prefix = "Simple Moving Average Step: ",
            xanchor = "left",
        ),

        x = 0,
        y = 0,
        len = 1,
        pad = dict(t = 0, b = 0),
        yanchor = "bottom",
        xanchor = "left",
    )

    for i in range((200 // 5) + 1):

        step = dict(
            method = "restyle",
            label = str(i * 5),
            value = str(i * 5),
            args = ["visible", [False] * ((200 // 5) + 1)],
        )

        step['args'][1][0] = True
        step['args'][1][i] = True
        sliders["steps"].append(step)



    layout = dict(

        title = 'Price of {}'.format(str(stock),
            title_font_size = 16),

        # ANIMATIONS
        sliders = [sliders],
        xaxis = dict(

            rangeselector = dict(
                activecolor = "#888888",
                bgcolor = "#DDDDDD",
                buttons = [
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD"),
                ]
            ),

        ),
        yaxis = dict(
            tickprefix = "$", tickformat = ',.',
            type = "linear",
            domain = [0.25, 1],
        ),

    )


    fig.update_layout(xaxis_rangeslider_visible = False)
    fig.layout(legend = list(orientation = 'h'))
    
    fig = go.Figure(data = data, layout = layout)



#     #Show
#     fig.show()
    pyo.iplot(fig)

In [ ]:
def candlestick_moving_average_50_200():
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=[
            'Price of {}'.format(str(stock)),
            'Volume of {}'.format(str(stock))])

    # Lineplots of price and moving averages
    fig.add_trace(go.Candlestick(
                x = df.index,
                open = df["Open"],
                high = df["High"],
                low = df["Low"],
                close = df["Close"],
                name = stock), row = 1, col = 1)
    
    fig.add_trace(go.Scatter(x = df.index,
                             y = df['short_SMA'],
                             name = 'Short SMA 50-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Short (50-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="blue")), row = 1, col = 1)

    fig.add_trace(go.Scatter(x = df.index,
                             y = df['long_SMA'],
                             name = 'Long SMA 200-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Long (200-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="black")), row = 1, col = 1)
    # Barplot of volume
    fig.add_trace(go.Bar(x = df.index,
                    y = df['Volume'],
                    name = 'Volume',
                    customdata = df['Name'],
                    hovertemplate="<b>%{customdata}</b><br><br>" +
                                    "Date: %{x|%d %b %Y} <br>" +
                                    "Volume: %{y:,.}<br>" +
                                    "<extra></extra>",
                    marker = dict(color="black")), row = 2, col = 1)
    # Add title
    fig.update_layout(
            title = 'Summary of {}'.format(str(stock)),
            title_font_size = 16,
            autosize = False, width = 800, height = 1000)
    
    fig['layout']['yaxis1']['title']='US Dollars'
    fig['layout']['yaxis2']['title']='Volume'
    

# X-Axes
    fig.update_xaxes(
        rangeselector = dict(
            buttons = list([
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD")])))
    fig.update_layout(xaxis_rangeslider_visible = False)
    fig.update_xaxes(rangeslider= {'visible':False}, row=2, col=1)
    fig.update_layout(xaxis_showticklabels=True, xaxis2_showticklabels=True)
    
    #time buttons
    fig.update_yaxes(tickprefix = '$', tickformat = ',.', row = 1, col = 1)
    fig.update_xaxes(rangeselector = {'visible' :False}, row = 2, col = 1)
    
    fig.update_layout(showlegend=False)

    pyo.iplot(fig)


In [ ]:
# creating graph for Close Percentage Change
def create_hist_and_box_pct_change():
    fig = make_subplots(rows=2, cols=1,
                        subplot_titles=['Histogram of {} 1-Day Close Percentage Change'.format(stock),
                                        'Box plot of {} 1-Day Close Percentage Change'.format(stock)],
                        x_title = '1-Day Close Percentage Change')
    # 1.Histogram
    fig.add_trace(go.Histogram(x = y['Close Percentage Change'], name = 'Histogram', nbinsx = round(len(y) / 20),
                               ), row=1, col=1)

    #2. Boxplot
    fig.add_trace(go.Box(x = y['Close Percentage Change'], name = 'Boxplot',
                         customdata = y['Name'],
                         hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "1-Day Percentage Change: %{x:.0%}<br>"+
                                    "<extra></extra>"
                    ), row=2, col=1)

    fig.update_layout(title = 'Plots of 1-Day Close Percentage Change for {}'.format(stock))
    fig['layout']['yaxis1']['title'] = '# of Observations'

    fig.update_xaxes(tickformat = '.0%', row = 1, col = 1)
    fig.update_xaxes(tickformat = '.0%', row = 2, col = 1)
    fig.update_layout(showlegend=False)

    pyo.iplot(fig)
    

In [ ]:
def returns():
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, subplot_titles=[
            'Closing Price of {}'.format(str(stock)),
            'Daily Return of {}'.format(str(stock)),
            'Monthly Return of {}'.format(str(stock)),
            'Annual Return of {}'.format(str(stock))])
    fig.add_trace(go.Scatter(
                            x = df.index,
                            y = df['Close'],
                            mode='lines',
                            customdata = df['Name'], name = 'Closing Price',
                            hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Closing Price: %{y:$,.2f}<br>"+
                                            "<extra></extra>"), row = 1, col = 1)

    fig.add_trace(go.Scatter(
                            x = df.index,
                            y = df['daily_return'],
                            mode='lines',
                            customdata = df['Name'], name = 'Daily Return',
                            hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Daily Return: %{y:,.0%}<br>"+
                                            "<extra></extra>"), row = 2, col = 1)

    fig.add_trace(go.Scatter(
                            x = df.index,
                            y = df['monthly_return'],
                            mode='lines',
                            customdata = df['Name'], name = 'Monthly Return',
                            hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Monthly Return: %{y:,.0%}<br>"+
                                            "<extra></extra>"), row = 3, col = 1)

    fig.add_trace(go.Scatter(
                            x = df.index,
                            y = df['annual_return'],
                            mode='lines',
                            customdata = df['Name'], name = 'Annual Return',
                            hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Annual Return: %{y:,.0%}<br>"+
                                            "<extra></extra>"), row = 4, col = 1)

    # Add titles
    fig.update_layout(
            title = 'Price of {}'.format(str(stock)),
            autosize = False, width = 800, height = 1000)
    fig['layout']['yaxis1']['title']='US Dollars'
    fig['layout']['yaxis2']['title']='% Return'
    fig['layout']['yaxis3']['title']='% Return'
    fig['layout']['yaxis4']['title']='% Return'
    # X-Axes
    fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD")])))
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_xaxes(rangeslider= {'visible':False}, row=2, col=1)
    fig.update_xaxes(rangeslider= {'visible':False}, row=3, col=1)
    fig.update_xaxes(rangeslider= {'visible':False}, row=4, col=1)

    fig.update_xaxes(rangeselector= {'visible':False}, row=2, col=1)
    fig.update_xaxes(rangeselector= {'visible':False}, row=3, col=1)
    fig.update_xaxes(rangeselector= {'visible':False}, row=4, col=1)

    fig.update_yaxes(tickprefix = '$', tickformat = ',.', row = 1, col = 1)
    fig.update_yaxes(tickformat = ',.000%', row = 2, col = 1)
    fig.update_yaxes(tickformat = ',.000%', row = 3, col = 1)
    fig.update_yaxes(tickformat = ',.000%', row = 4, col = 1)

    fig.update_layout(showlegend=False)
    #Show
    pyo.iplot(fig)

In [ ]:
def diff_plot():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = y.index,
                            y = y['diff'],
                            name = str(stock), 
                            mode='lines',
                            customdata = y['Name'],
                            hovertemplate="<b>%{customdata}</b><br><br>" +
                                    "Date: %{x|%d %b %Y} <br>" +
                                    "Price Volatility: %{y:$,.2f}<br>"+
                                    "<extra></extra>"))
    # Add titles
    fig.update_layout(
        title = 'Difference in price of Lag(1) for {}'.format(stock),
        yaxis_title = 'US Dollars',
        yaxis_tickprefix = '$', yaxis_tickformat = ',.')
    # X-Axes
    fig.update_xaxes(
        rangeslider_visible = False,
        rangeselector = dict(
            buttons = list([
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD")])))
    fig.update_layout(showlegend=False)
    #Show
    pyo.iplot(fig)
    

In [ ]:
# ZOOMING IN
# https://community.plotly.com/t/y-axis-autoscaling-with-x-range-sliders/10245/4
import plotly.graph_objs as go 


trace =  go.Candlestick(
                x = df.index,
                open = df["Open"],
                high = df["High"],
                low = df["Low"],
                close = df["Close"],
                name = stock)
short_sma = go.Scatter(x = df.index,
                             y = df['short_SMA'],
                             name = 'Short SMA 50-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Short (50-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="blue"))

long_sma = go.Scatter(x = df.index,
                             y = df['long_SMA'],
                             name = 'Long SMA 200-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Long (200-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="black"))

data = [trace, short_sma, long_sma]
layout = dict(
    title = 'Price of {} (zoom in)'.format(stock),
    autosize = False, width = 800, height = 600,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD")])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = go.FigureWidget(data=data, layout=layout)
# fig.update_layout(showlegend=False)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=-0.5,
    xanchor="right",
    x=1
))

def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.Close.min() - 0.25, in_view.Close.max() + 0.25]

fig.layout.on_change(zoom, 'xaxis.range')
fig

In [ ]:
# ZOOMING IN
# https://community.plotly.com/t/y-axis-autoscaling-with-x-range-sliders/10245/4
import plotly.graph_objs as go 

trace = go.Bar(x = df.index,
                    y = df['Volume'],
                    name = 'Volume',
                    customdata = df['Name'],
                    hovertemplate="<b>%{customdata}</b><br><br>" +
                                    "Date: %{x|%d %b %Y} <br>" +
                                    "Volume: %{y:,.}<br>" +
                                    "<extra></extra>",
                    marker = dict(color="black"))

data = [trace]
layout = dict(
    title = 'Volume of {} (zoom in)'.format(stock),
    autosize = False, width = 800, height = 600,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD")])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = go.FigureWidget(data=data, layout=layout)
fig.update_layout(showlegend=False)

def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.Volume.min() - 0.25, in_view.Volume.max() + 500000]

fig.layout.on_change(zoom, 'xaxis.range')
fig

### Graphs need Work